<img src="https://github.com/Microsoft/azuredatastudio/blob/main/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="10%">

## SQL Notebooks in Azure Data Studio

Notebooks allow rich text, images, code, and resultsets to be easily shared. This is a concept that is widely used in data science and which we feel is well suited to SQL work. 


In [1]:
select 
    'Hello SQL World' as [Greetings], 
    @@servername as [Server Name],
    datename(weekday,getutcdate()) as [Today]

### Concepts

Notebooks are saved in a file format of .ipynb and have a couple of logical components.

**Kernel**

The language and execution environment of the notebook. Common examples are Python and PowerShell. Azure Data Studio also offers a SQL kernel, which is the focus of this tutorial.

**Attach To**

This is the compute environment for the code - basically, where it will run. For SQL this is exactly analogous to the familiar Connection property of a query.

**Cell**

A cell is an editable section of the notebook. Cells can be human-readable text or code. Text cells are edited in the Markdown language to allow formatting and can include rich content including images. In Azure Data Studio, code cells include intellisense where possible. Below is an example of a SQL code cell.


In [7]:
select top 5 * from sys.dm_exec_session_wait_stats order by wait_time_ms desc

(5 rows affected)

Total execution time: 00:00:00.163

session_id,wait_type,waiting_tasks_count,wait_time_ms,max_wait_time_ms,signal_wait_time_ms
59,ASYNC_NETWORK_IO,24,389,347,0
57,PREEMPTIVE_XE_GETTARGETSTATE,9,160,102,0
57,ASYNC_NETWORK_IO,323,38,2,10
58,MEMORY_ALLOCATION_EXT,5066,25,0,0
60,PAGEIOLATCH_SH,30,11,0,0


**Results**

Code cells can be executed against the environment specified in Attach To, and their results are saved into the notebook. Once executed and saved, the results can be opened with the file without having be be re-executed. Results may be text, grids, charts, or other visualizations.

**Trusted vs Non-Trusted**

Since notebooks can contain HTML and Javascript code, it would be possible for a maliciously formed notebook to execute code simply upon being opened by the user. To prevent this, notebooks have the concept of "Trusted" and "Untrusted." Untrusted HTML is sanitized and untrusted Javascript is not executed. Users can mark notebooks trusted if necessary, but in common use cases an untrusted notebook will perform the same as a trusted one. This is similar to the behavior of many other file types when downloaded from the internet.

### Editing Experience

Editing SQL is the same in Azure Data Studio notebooks as in the Azure Data Studio query editor - the same Intellisense, the same snippet support. Markdown is edited with a custom editor which shows a preview during edit, and only shows the processed markdown when not in edit mode. Code and markdown (text) cells can be added with buttons or through context menus.

In [1]:
select top 10 * from sys.databases

### Code Separation and Context

The separate code cells in a SQL notebook are run as separate batches. This is equivalent to having a GO statement between cells.

This is an area we're actively innovating in and want to make a little more straightforward! But we'll describe the current behavior.

In this code, we're setting the database context to WideWorldImporters and then creating a local variable. We're also setting nocount on, to demonstrate set statement behavior.

In [1]:
set nocount on;
use WideWorldImporters;
declare @i int;
set @i = 1;
select @i [Value of @i], db_name() as [MyDatabaseName]

The next cell will show an error, since the variable @i is not declared within this cell.

In [1]:
select @i [Value of @i], db_name() as [MyDatabaseName]

In the next cell, we're re-declaring @i, but not setting it, so the value remains null.

However, the previously set database context and nocount settings are maintained as these are durable throughout the session. This is the same behavior as the query editor.

In [1]:
declare @i int;
select @i [Value of @i], db_name() as [MyDatabaseName]

One upshot of this - not unique to SQL notebooks - is that code cell execution order *does matter*

We think this is an area we can innovate in and welcome your suggestions on how to make this easier to work with in a notebook context.

### Multi-language support

While in a SQL notebook you can change to run R and Python on the SQL Server with a special syntax. This allows you to type your raw Python or R and execute it without having to remember how to wrap it up in a stored procedure (provided that the Advanced Analytics Extensions feature is installed and enabled on the target server)

In [1]:
%%lang_python
print ("Hello World")

STDOUT message(s) from external script: 
Hello World

Total execution time: 00:00:21.211

### Use Cases

We're envisioning a number of use cases for the SQL notebooks. In many ways it's simply the next generation experience for the query editor, and many of the things you can do in a plain editor can be done as well or better in a notebook. But a few scenarios have jumped out at us:

- Teaching Tools 
- Runbooks
- Business reporting and presentations
- Troubleshooting
- Deployments
- Baselining

We're excited to learn what the community can do with them!

**Example Server Information Report**

Run this report and save it to a known location to maintain a basic point in time inventory.

In [1]:
--Server basics
SELECT
    getutcdate() as DateRun, 
    db_name() as DatabaseName,
    serverproperty('ServerName') as ServerName,
    serverproperty('InstanceName') as InstanceName,
    serverproperty('ComputerNamePhysicalNetBIOS') as PhysicalName,
    serverproperty('Edition') as Edition,
    serverproperty('ProductMajorVersion') as MajorVersion,
    serverproperty('ProductMinorVersion') as MinorVersion

--Databases
SELECT *
from sys.databases

**More SQL Examples**



In [1]:
DECLARE @i INT,  @c varchar(26);
SELECT @i = 65, @c = '';
WHILE (@i < 93)
BEGIN
    SELECT @c = concat(@c,CHAR(@i))
    SET @i = @i + 1
END


SELECT @c as "Letters", len(@c) as "Count"

In [1]:
DROP TABLE IF EXISTS [dbo].[MyNotebookTable]
CREATE TABLE [dbo].[MyNotebookTable]
(
    [Id] INT IDENTITY NOT NULL PRIMARY KEY, -- Primary Key column
    [FirstValue] NVARCHAR(50) NOT NULL,
    [SecondValue] NVARCHAR(50) NOT NULL
);

PRINT 'Success: Created MyNotebookTable'


In [1]:
raiserror('Something bad happened!',0,1) with nowait;
waitfor delay '00:00:05'
raiserror('Something bad happened... again!',0,1) with nowait;